# 1. install Openai-agents SDK

In [1]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.2 MB/s eta 0:00:00


## Make your Jupyter Notebook capable of running asynchronous functions.

In [2]:
import nest_asyncio
nest_asyncio.apply()

### Get api keys

In [14]:
import os
from google.colab import userdata
gemini_api_key = userdata.get("GEMINI_API_KEY")

# 2. Run Google Gemini with OPENAI-Agent SDK

In [15]:
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel
from agents.run import RunConfig

In [22]:
# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client,
)

config = RunConfig(
 model=model,
 model_provider=external_client,
 tracing_disabled=True,
)


### **AsyncOpenAI**: ye class `openai-agents SDK` ke ander kisi dosre LLM ko integret karne ke lea use hoti hai.

### **OpenAIChatCompletionsModel**: is class mien hum ye btate hai ke hum kon sa model use kar rahe hai aur us ka provider kon hai.

### **RunConfige**: is mien hum model aur us ke provider aur OpenAI-Agents SDK ke new features pass kar sakhte hai jaise: `tracing_disabled=True,`configuration jo jab bhi hum agent ko run kare toh us waqt kaam ae gi.

# 3. **Hello world code | method one (syncronously)**

In [29]:
agent: Agent = Agent(name="Assistant", instructions="You are a helpful assistant", model=model)
userinput = input('Enter your question: ')
result = Runner.run_sync(agent, input=userinput, run_config=config)

print("\nCALLING AGENT\n")
print(result.final_output)

Enter your question: Who is the Founder of Pakistan?

CALLING AGENT

The founder of Pakistan is generally considered to be **Muhammad Ali Jinnah**.



# **4. Hello world code | method two (asyncronously)**:

In [31]:
import asyncio
from agents import Agent, Runner

async def main():
  agent: Agent = Agent(name="Assistant", instructions="You only respond in haikus.", model=model)
  userinput = input('Enter your question: ')

  result = await Runner.run(agent, input=userinput, run_config=config)
  # Runner mien .run method by defualt async hota hai.
  print("\nCALLING AGENT\n")
  print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

Enter your question: Tell me about recursion in programming

CALLING AGENT

A function calls self,
Repeating 'til the base case,
Problem shrinks down small.



# 5. **Hello world code | method three (streaming):**

### **1. Streaming Text code:**
is mien hum sirf text ko stream kar rahe hai.

In [40]:
import asyncio
from openai.types.responses import ResponseTextDeltaEvent
from agents import Agent, Runner


async def main():
    agent = Agent(
        name="Joker",
        instructions="You are a helpful assistant.",
        model=model
    )

    result = Runner.run_streamed(agent, input="Please tell me 5 jokes.")
    async for event in result.stream_events():
      if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
          print(event.data.delta, end="", flush=True)



asyncio.run(main())

Okay, here are 5 jokes for you:

1.  Why don't scientists trust atoms? Because they make up everything!

2.  What do you call a lazy kangaroo? Pouch potato!

3.  Why did the scarecrow win an award? Because he was outstanding in his field!

4.  Parallel lines have so much in common. It’s a shame they’ll never meet.

5.  Why don’t eggs tell jokes? They’d crack each other up!

Hope you enjoyed them!


Yeh code ek asynchronous streaming response handle karne ka tareeqa hai, jahan pe jokes ka jawab stream ki shakal mein aata hai. Aaiye, har line ko asaan alfaaz mein samajhte hain:

1. **Result ko stream karna:**
   ```python
   result = Runner.run_streamed(agent, input="Please tell me 5 jokes.")
   ```
   - **Kya ho raha hai:** Yeh line `Runner.run_streamed` function ko call kar rahi hai, jismein agent (aapka assistant) ko input message diya gaya hai: "Please tell me 5 jokes." Iska matlab hai ke aap assistant se 5 mazedaar chutkule sunna chahte hain.
   - **Output:** Yeh function ek aisa result return karta hai jo stream (data ka chalta hua silsila) ke roop mein hota hai.

2. **Stream events ko asynchronously iterate karna:**
   ```python
   async for event in result.stream_events():
   ```
   - **Kya ho raha hai:** Yeh loop asynchronous tareeke se stream events (mukhtalif hisse jo response mein aate hain) ko ek ke baad ek process karta hai. Matlab, jaise hi assistant ka jawab thoda thoda karke aata hai, yeh loop usay handle karta hai.

3. **Event type aur data check karna:**
   ```python
   if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
   ```
   - **Kya ho raha hai:** Har event ko check kiya ja raha hai:
     - **`event.type == "raw_response_event"`:** Yeh condition ensure karti hai ke event ka type raw response hai, matlab asli text ka hissa.
     - **`isinstance(event.data, ResponseTextDeltaEvent)`**: Yeh confirm karta hai ke event ka data, text ka ek chhota hissa (delta) hai.
   - **Matlab:** Sirf un events ko process karo jo text ke chhote chhote tukdo (delta) mein response provide kar rahe hon.

4. **Response text ko print karna:**
   ```python
   print(event.data.delta, end="", flush=True)
   ```
   - **Kya ho raha hai:** Yeh line event ke data se text ka hissa (delta) print kar rahi hai:
     - **`end=""`:** Matlab print ke baad newline na lagayein, taake text lagataar (streaming) print hota rahe.
     - **`flush=True`:** Yeh ensure karta hai ke print output turant screen par dikh jaye.
   - **Matlab:** Jaise hi assistant ka jawab aata hai, woh turant print hota jata hai, bina delay ke.

**Overall Concept:**

- **Asynchronous Streaming:** Yeh code response ko ek dam ek sath (real-time) stream karta hai, jismein assistant ka jawab chhote chhote hisson mein aata hai aur turant print hota jata hai.
- **Event Filtering:** Sirf un events ko process karta hai jo raw text responses hain, is se aapko sahi aur expected output milta hai.

Agar koi aur sawaal ho ya koi aur detail chahiye, to aap pooch sakte hain!

### **2. Streaming item code:**
is mien hum text ke sath jo bi data ho ga us sab ko stream karen ge

In [47]:
import asyncio
import random

from agents import Agent, ItemHelpers, Runner, function_tool


@function_tool
def how_many_jokes() -> int:
    return random.randint(1, 10)


async def main():
    agent = Agent(
        name="Joker",
        instructions="First call the `how_many_jokes` tool, then tell that many jokes.",
        tools=[how_many_jokes],
        model=model
    )

    result = Runner.run_streamed(
        agent,
        input="Hello",

    )
    print("=== Run starting ===")
    async for event in result.stream_events():
        # We'll ignore the raw responses event deltas
        if event.type == "raw_response_event":
            continue
        elif event.type == "agent_updated_stream_event":
            print(f"Agent updated: {event.new_agent.name}")
            continue
        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                print("-- Tool was called")
            elif event.item.type == "tool_call_output_item":
                print(f"-- Tool output: {event.item.output}")
            elif event.item.type == "message_output_item":
                print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
            else:
                pass  # Ignore other event types



asyncio.run(main())

print("=== Run complete ===")

=== Run starting ===
Agent updated: Joker
-- Tool was called
-- Tool output: 7
-- Message output:
 Ok, I will tell you 7 jokes.

Why don't scientists trust atoms?
Because they make up everything!

What do you call a lazy kangaroo?
Pouch potato!

Why did the bicycle fall over?
Because it was two tired!

Have you heard about the restaurant on the moon?
I heard the food was good but it had no atmosphere.

Why did the scarecrow win an award?
Because he was outstanding in his field!

Why did the teddy bear say no to dessert?
Because she was stuffed.

What musical instrument is found in the bathroom?
A tuba toothpaste.

=== Run complete ===
